In [21]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from main import init_env_vars
from langchain import FAISS
from langchain.embeddings import OpenAIEmbeddings

init_env_vars()

db = FAISS.load_local('index', OpenAIEmbeddings())
retriever = db.as_retriever(search_type='similarity', search_kwargs={'k': 4})


def ask_bot(question, ai_instruction):
    system_template = ai_instruction + "\n\n{summaries}"
    messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
    chain_type_kwargs = {"prompt": ChatPromptTemplate.from_messages(messages)}
    qa = RetrievalQAWithSourcesChain.from_chain_type(
        llm=ChatOpenAI(temperature=0),
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs, verbose=True)
    result = qa({"question": question})
    return result

In [22]:
def get_sources(result):
    source_urls = []
    for document in result['source_documents']:
        url = document.metadata['source'].replace("website/", "https://")
        if url not in source_urls:
            source_urls.append(url)

    return source_urls

In [23]:
# ai_instruction = "Give your answer in a bullet point format to be used for slide presentations"
ai_instruction = ""
question = "What are the different retrieval chain I can use for question and answer bot?"

response = ask_bot(question, ai_instruction)

print(response['answer'])
print(get_sources(response))



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
There are several retrieval chains that you can use for a question and answer bot, depending on your specific needs and preferences. Here are a few examples:

1. ConversationalRetrievalChain: This chain condenses the current question and chat history into a standalone question, performs retrieval, and then answers the question using retrieval augmented generation with a separate model.

2. RetrievalQA: This chain performs retrieval on a given index and then answers the question using a language model.

3. GraphCypherQAChain: This chain performs retrieval on a graph database using Cypher queries and then answers the question using a language model.

4. MultiRetrievalQAChain: This chain allows you to select from multiple retrieval models and then answers the question using a language model.

5. FLARE: This chain uses a combination of retrieval and generation to answer questions, with a focus on explainability and in

In [24]:
from langchain.chains import ConversationalRetrievalChain


def ask_bot_conversational(question):
    retriever = db.as_retriever(search_type='similarity', search_kwargs={'k': 4})
    chat_history = []
    qa = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(temperature=0), retriever=retriever, verbose=True, return_source_documents=True)

    return qa({"question": question, "chat_history": chat_history})

In [25]:
question = "What is StuffDocumentsChain chain?"
result = ask_bot_conversational(question)

print(result['answer'])
print(get_sources(result))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
config

dict

*

kwargs

Any

>

StuffDocumentsChain

if

"llm_chain"

in

config

llm_chain_config

config

pop

"llm_chain"

llm_chain

load_chain_from_config

llm_chain_config

elif

"llm_chain_path"

in

config

llm_chain

load_chain

config

pop

"llm_chain_path"

))

else

raise

ValueError

"One of `llm_chain` or `llm_chain_config` must be present."

if

not

isinstance

llm_chain

LLMChain

):

raise

ValueError

"Expected LLMChain, got

llm_chain

if

"document_prompt"

in

config

prompt_config

config

pop

"document_prompt"

document_prompt

load_prompt_from_config

prompt_config

elif

"document_prompt_path"

in

config

document_prompt

load_prompt

config

pop

"document_prompt_path"

)

In [26]:
question = "How to use ChatPromptTemplate?"
result = ask_bot_conversational(question)

print(result['answer'])
print(get_sources(result))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Chat Prompt Templates#

Chat Models take a list of chat messages as input - this list commonly referred to as a prompt.
These chat messages differ from raw string (which you would pass into a LLM model) in that every message is associated with a role.

For example, in OpenAI Chat Completion API, a chat message can be associated with the AI, human or system role. The model is supposed to follow instruction from system chat message more closely.

LangChain provides several prompt templates to make constructing and working with prompts easily. You are encouraged to use these chat related prompt templates instead of PromptTemplate when querying chat models to fully exploit the potential of underlying chat